Übung: 3 VMs zu einem Kubernetes Cluster verbinden
--------------------------------------------------

MicroK8s bietet eine einfache und effiziente Möglichkeit, Kubernetes-Cluster lokal oder in Produktionsumgebungen zu betreiben. 

Das Hinzufügen von Nodes zu einem bestehenden MicroK8s-Cluster erfolgt über den Befehl `microk8s add-node`. 

Dieser Befehl generiert die notwendigen Verbindungsinformationen und Token, die benötigt werden, um neue Nodes (Master oder Worker) sicher in den Cluster zu integrieren.

- - -

Um die Vorgänge zu visualisieren starten wird Kubernetes Dashboard und scrollen rechts herunter zu Cluster -> Nodes

Wählt dazu nachfolgenden Link an und aktzeptiert das Zertifikat um dann ohne Token, drückt "Überspringen" oder "Skip", ins Dashboard zu wechseln.


In [ ]:
! echo "https://"$(cat ~/work/server-ip)":18443"

In [ ]:
%%bash
# Verbindungsinformationen und Token aufbereiten
export JOIN=$(microk8s add-node | grep worker | tail -1)
echo $JOIN
# Knoten (Worker) 01 integrieren
ssh dukworker-11-default -- ${JOIN}

In [ ]:
%%bash
# Knoten (Worker) 02 integrieren
export JOIN=$(microk8s add-node | grep worker | tail -1)
ssh dukworker-12-default -- ${JOIN}

In [ ]:
! kubectl get nodes 

Mittels der Label (Unterschiedlich pro Distribution) können wir die Rollen: **Master** oder **Worker** anzeigen lassen

In [ ]:
%%bash
echo "Master"
kubectl get nodes -l node.kubernetes.io/microk8s-controlplane=microk8s-controlplane
echo "--------------------------------------------------------"
echo "Worker"
kubectl get nodes -l node.kubernetes.io/microk8s-worker=microk8s-worker

- - -

### Zentraler Persistenter Speicher

Damit alle Kubernetes Nodes (Master + Worker) den gleichen zentralen Persistenten Speicher verwenden, wurder vorher auf dem Master ein NFS Share /data eingerichtet.

In [ ]:
! cat /etc/exports

Damit müssen sich jetzt noch die Worker Nodes verbinden

In [ ]:
! ssh dukworker-11-default -- "sudo mount -t nfs dukmaster-10-default:/data /data; df -h | grep /data"

In [ ]:
! ssh dukworker-12-default -- "sudo mount -t nfs dukmaster-10-default:/data /data; df -h | grep /data"

Die entsprechende Dateiablage, sogenannte PersistentVolumes (PV) in Kubernetes, wurde beim Installieren von Kubernetes eingerichtet. Das PersistentVolumes zeigt auf das Verzeichnis `/data`, siehe Eintrag `hostPath`.

Diese Einrichtung ermöglicht die Trennung von Speicher und Container-Lebenszyklus, sodass Daten über Container-(Pod)-Neustarts und -Löschungen hinaus bestehen bleiben.

In [ ]:
! kubectl get persistentvolumes -o yaml | kubectl-neat

Im Kubernetes Dashboard finden wir den Eintrag in dem wir rechts herunterscrollen auf Cluster -> PersistentVolumes

In [ ]:
! echo "https://"$(cat ~/work/server-ip)":18443"